# 1. 유튜브 인기 동영상 데이터

In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv",index_col=0)


In [2]:
df.head(5)

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01


## 1.1 논란으로 인기동영상이 된 케이스 확인 -> dislike 수가 like 수보다 높은 동영상을 제작한 채널 출력

In [3]:
answer = df[df['dislikes']>df['likes']]['channelTitle'].unique()
print(answer)

['핫도그TV' 'ASMR 애정TV' '하얀트리HayanTree' '양팡 YangPang' '철구형 (CHULTUBE)'
 '왜냐맨하우스' '(MUTUBE)와꾸대장봉준' '오메킴TV' '육지담' 'MapleStory_KR'
 'ROAD FIGHTING CHAMPIONSHIP' '사나이 김기훈' '나혼자산다 STUDIO' 'Gen.G esports']


## 1.2 채널명을 바꾼 케이스가 있는지 확인 -> channelId의 경우 고유값이므로 이를 통해 채널명을 한번이라도 바꾼 채널의 갯수

In [4]:
df.head(1)

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01


In [5]:
change = df[['channelTitle', 'channelId']].drop_duplicates(keep='first')['channelId'].value_counts()
answer = len(change[change>1])
print(answer)

71


## 1.3 일요일에 인기있었던 영상들 중 가장 많은 영상 종류는 무엇인가?

In [6]:
df.head(3)

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01


In [7]:
df['trending_date2'] = pd.to_datetime(df['trending_date2'])
answer = df[df['trending_date2'].dt.day_name() == 'Sunday']['categoryId'].value_counts().index[0]
print(answer)

24


## 1.4 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터프레임으로 표현하라 

In [8]:
df['trending_date2'] = pd.to_datetime(df['trending_date2'])
group = df.groupby([df['trending_date2'].dt.day_name(), 'categoryId'], as_index = False).size()
answer = group.pivot(index='categoryId', columns='trending_date2')
print(answer)

                 size                                                  
trending_date2 Friday Monday Saturday Sunday Thursday Tuesday Wednesday
categoryId                                                             
1                 243    263      255    274      246     257       234
2                 120    105      119     99      128     119       129
10                833    837      776    830      890     894       917
15                187    215      198    217      207     208       207
17                633    668      592    636      682     708       706
19                 90     92       87     91       92      89        85
20                283    298      296    289      282     285       291
22               1288   1373     1289   1337     1341    1375      1333
23                568    594      570    556      560     569       566
24               2976   3148     3066   3096     2954    3084      3090
25                444    453      422    437      470     452   

## 1.5 댓글의 수로(comment_count) 영상 반응에 대한 판단을 할 수 있다. viewcount 대비 댓글 수가 가장 높은 영상 확인 (view_count값 0인 경우 제외)

In [9]:
drop_index = df[df['view_count']==0].index
df_ = df.drop(drop_index, axis=0)
df_['value'] = df_['comment_count']/df_['view_count']
df_ = df_.sort_values('value', ascending=False)
answer = df_.iloc[0,0]
print(answer)

60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)


## 1.6 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다. view_count 대비 댓글 수가 가장 낮은 영상 확인하라 (view_counts, ratio 값이 0인 경우 제외)

In [10]:
drop_idx = df[df['view_count']==0].index
df_ = df.drop(drop_idx, axis=0)
df_['ratio'] = df['comment_count']/df['view_count']
drop_idx2 = df_[df_['ratio']==0].index
df_.drop(drop_idx2, axis=0, inplace=True)
df_ = df_.sort_values('ratio')
answer = df_.iloc[0,0]
print(answer)

Join the BTS #PermissiontoDance Challenge only on YouTube #Shorts


## 1.7 like 대비 dislike 수가 가장 적은 영상 무엇인가? (like, dislike 값이 0인 경우 제외)

In [11]:
idx1 = df[df['likes']==0].index
df_ = df.drop(idx1, axis=0)
idx2 = df_[df_['dislikes']==0].index
df_ = df_.drop(idx2, axis=0)
df_['ratio'] = df_['dislikes']/df['likes']
df_ = df_.sort_values('ratio')
answer = df_.iloc[0,0]
print(answer)

[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰


## 1.8 가장 많은 트렌드 영상을 제작한 채널의 이름은 무엇인가? (날짜기준, 중복포함)

In [12]:
df.head(2)

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01


In [13]:
answer = df[df['channelId'] == df['channelId'].value_counts().index[0]]['channelTitle'].unique()
print(answer)

['짤툰']


## 1.9 20회 (20일) 이상 인기동영상 리스트에 포함된 동영상의 숫자는?

In [14]:
answer = sum(df.groupby(['title','channelId'])['trending_date2'].count()>=20)
print(answer)

40


In [15]:
answer = (df[['title', 'channelId']].value_counts()>=20).sum()
print(answer)

40


# 2. 유튜브 공범컨텐츠 동영상 데이터

In [16]:
import pandas as pd


channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')

In [17]:
channel.head(2)

,channelid,subcnt,viewcnt,videocnt,ct,channelname
0,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,410238653,736,2021-09-30 03:01:03,꽈뚜룹
1,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,736,2021-09-30 09:01:03,꽈뚜룹


In [18]:
video.head(2)

,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
0,c5JQp6xafqc,1667010,30474,706,0,6587,2021-10-10 15:20:03,공범 EP1
1,c5JQp6xafqc,1669089,30495,707,0,6589,2021-10-10 15:30:03,공범 EP1


## 2.1 각 데이터의 'ct' 컬럼을 시간으로 인식할 수 있게 datetype으로 변경하고 video 데이터의 videoname의 각 value 마다 몇 개의 데이터를 가지고 있는지 확인

In [19]:
channel['ct'] = pd.to_datetime(channel['ct'])
video['ct'] = pd.to_datetime(video['ct'])
answer = video['videoname'].value_counts()
print(answer)

 공범 EP1    3492
 공범 EP2    3204
 공범 EP3    2568
 공범 EP4    2280
 공범 EP5    1562
 공범 EP6    1274
 공범 EP7     555
 공범 EP8     266
Name: videoname, dtype: int64


## 2.2 수집된 각 video의 가장 최신화 된 날짜의 viewcount값을 출력하라

In [20]:
answer = video.sort_values(['videoname', 'ct']).drop_duplicates('videoname', keep='last')[['viewcnt', 'ct', 'videoname']]
print(answer)

       viewcnt                  ct videoname
3491   3180532 2021-11-01 15:30:03    공범 EP1
6695   2199328 2021-11-01 15:30:03    공범 EP2
9263   1671294 2021-11-01 15:30:03    공범 EP3
11543  1818493 2021-11-01 15:30:03    공범 EP4
13105  1503435 2021-11-01 15:30:04    공범 EP5
14934  1750222 2021-11-01 15:30:04    공범 EP6
13660  1630200 2021-11-01 15:30:05    공범 EP7
15200  1289088 2021-11-01 15:30:05    공범 EP8


## 2.3 channel 데이터 중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수 (subcnt) 를 출력하라 

In [21]:
channel_ = channel[channel['ct']>=pd.to_datetime('2021-10-03')]
channel_ = channel_[['channelname', 'subcnt']]
channel_ = channel_.drop_duplicates('channelname', keep='first')
print(channel_)

          channelname   subcnt
12                꽈뚜룹  1330000
139             야전삽짱재   257000
266        츄정ChuJeong   322000
393            논리왕 전기   922000
520   와글와글 WagleWagle    55000
647               조나단   215000
774           형사!탐정되다    14900
901        김농밀의 농밀한 삶     7520
1028      릴펄 Lilpearl    10100
1155    곽토리 kwak tori   471000
1276    Balming Tiger    54300


## 2.4 각 채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수의 증가량을 구하여라

In [22]:
channel_ = channel[(channel['ct']>=pd.to_datetime('2021-10-03 03:00:00')) & (channel['ct']<=pd.to_datetime('2021-11-01 15:00:00'))]
channel_ = channel_[['channelname', 'subcnt']]
ch_1 = channel_.drop_duplicates('channelname', keep='first')
ch_2 = channel_.drop_duplicates('channelname', keep='last')
ch = pd.merge(ch_1, ch_2, on='channelname', how='inner')
ch['del'] = ch['subcnt_y']-ch['subcnt_x']
ch = ch[['channelname', 'del']]
print(ch)

        channelname    del
0               꽈뚜룹  70000
1             야전삽짱재  11000
2        츄정ChuJeong   1000
3            논리왕 전기 -11000
4   와글와글 WagleWagle      0
5               조나단  12000
6           형사!탐정되다  10300
7        김농밀의 농밀한 삶   1540
8       릴펄 Lilpearl  11000
9     곽토리 kwak tori  -2000
10    Balming Tiger   2500


## 2.5 각 비디오는 10분 간격으로 구독자수, 좋아요, 싫어요 수, 댓글 수가 수집된 것으로 알려졌다. 공범 EP1의 비디오 정보 데이터 중 수집간격이 5분 이하, 20분 이상인 데이터 구간의 시각을 모두 출력하라

In [23]:
import datetime
video_1 = video[video['videoname']==' 공범 EP1'].sort_values('ct').reset_index(drop=True)
video_1[(video_1['ct'].diff(1)>=datetime.timedelta(minutes=20)) | (video_1['ct'].diff(1)<=datetime.timedelta(minutes=5))]
answer = video_1[video_1.index.isin([720, 721, 722, 723, 1635, 1636, 1637])]
print(answer)

          videopk  viewcnt  likecnt  dislikecnt  favoritecnt  cmcnt  \
720   c5JQp6xafqc  2153676    34104         830            0   7120   
721   c5JQp6xafqc  2228250    34559         849            0   7191   
722   c5JQp6xafqc  2228250    34559         849            0   7191   
723   c5JQp6xafqc  2228669    34566         849            0   7194   
1635  c5JQp6xafqc  2706691    37893         987            0   7410   
1636  c5JQp6xafqc  2707933    37901         988            0   7410   
1637  c5JQp6xafqc  2708448    37903         988            0   7410   

                      ct videoname  
720  2021-10-12 19:20:03    공범 EP1  
721  2021-10-13 09:41:37    공범 EP1  
722  2021-10-13 09:41:37    공범 EP1  
723  2021-10-13 09:50:03    공범 EP1  
1635 2021-10-19 17:50:02    공범 EP1  
1636 2021-10-19 18:20:03    공범 EP1  
1637 2021-10-19 18:30:03    공범 EP1  


## 2.6 각 에피소드의 시작날짜 (년-월-일)를 에피소드 이름과 묶어 데이터프레임으로 만들고 출력하라

In [24]:
video_ = video[['ct', 'videoname']].sort_values('ct')
answer = video_.drop_duplicates('videoname', keep='first')
print(answer)

                       ct videoname
10    2021-10-07 19:10:03    공범 EP1
3496  2021-10-09 19:10:03    공범 EP2
6696  2021-10-14 19:10:04    공범 EP3
9264  2021-10-16 19:10:04    공범 EP4
11544 2021-10-21 19:10:04    공범 EP5
13661 2021-10-23 19:10:04    공범 EP6
13106 2021-10-28 19:10:04    공범 EP7
14935 2021-10-30 19:20:04    공범 EP8


## 2.7 '공범' 컨텐츠의 경우 19:00시에 공개되는 것으로 알려졌다. 공개된 날의 21시의 viewcnt, ct, videoname으로 구성된 데이터 프레임을 viewcnt를 내림차순으로 정렬하여 출력하라

In [25]:
video['time'] = video['ct'].dt.hour
answer = video[video['time']==21].sort_values(['videoname', 'ct']).drop_duplicates('videoname').sort_values('viewcnt', ascending=False)[['viewcnt', 'ct', 'videoname']].reset_index(drop=True)
print(answer)

   viewcnt                  ct videoname
0   264029 2021-10-30 21:00:08    공범 EP8
1   252032 2021-10-28 21:00:04    공범 EP7
2   217674 2021-10-16 21:00:04    공범 EP4
3   213899 2021-10-23 21:00:06    공범 EP6
4   201179 2021-10-21 21:00:04    공범 EP5
5   148144 2021-10-09 21:00:03    공범 EP2
6   147183 2021-10-14 21:00:04    공범 EP3
7   117340 2021-10-07 21:00:03    공범 EP1


## 2.8 video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고 videoname, ratio로 구성된 데이터 프레임을 ratio 오름차순으로 정렬하라

In [26]:
video_ = video[['videoname', 'likecnt', 'dislikecnt', 'ct']].sort_values('ct', ascending=False)
video_ = video_.drop_duplicates('videoname', keep='first')
video_['ratio'] = video_['dislikecnt']/video_['likecnt']
answer = video_[['videoname', 'ratio']].sort_values('ratio')
print(answer)

      videoname     ratio
13660    공범 EP7  0.013636
14934    공범 EP6  0.015881
11543    공범 EP4  0.017971
13105    공범 EP5  0.021128
9263     공범 EP3  0.022360
3491     공범 EP1  0.027118
6695     공범 EP2  0.029844
15200    공범 EP8  0.141357


## 2.9 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오

In [34]:
video_ = video[(video['ct']>=pd.to_datetime('2021-11-01 00:00:00'))&(video['ct']<=pd.to_datetime('2021-11-01 15:00:00'))]
video1 = video_[['videoname', 'viewcnt', 'ct']].drop_duplicates('videoname', keep='first')
video2 = video_[['videoname', 'viewcnt', 'ct']].drop_duplicates('videoname', keep='last')
video_ = pd.merge(video1, video2, on='videoname')
video_['del'] = video_['viewcnt_y']-video_['viewcnt_x']
answer = video_[['videoname', 'del']]
print(answer)

  videoname    del
0    공범 EP1  13298
1    공범 EP2  10300
2    공범 EP3   9927
3    공범 EP4   9824
4    공범 EP5  10824
5    공범 EP7  26949
6    공범 EP6  14141
7    공범 EP8  89147


# 2.10 video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와 videoname 을 구하여라

In [65]:
video_ = video[video.index.isin(set(video.index) - set(video.drop_duplicates().index))]
answer = video_[['videoname', 'ct']]
print(answer)

     videoname                  ct
722     공범 EP1 2021-10-13 09:41:37
3927    공범 EP2 2021-10-13 09:41:37
